Spotify API that pulls audio features of artists' top 10 songs

In [57]:
from dotenv import load_dotenv
import os
import base64
from requests import post, get
import json
import pandas as pd

In [58]:
load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [59]:
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes),"utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}
    result = post(url, headers = headers, data = data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

In [60]:
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [61]:
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query
    result = get(query_url, headers=headers)
    json_result = json.loads(result.content)["artists"]["items"]
    
    if len(json_result) == 0:
            print("N/A")
            return None

    return json_result[0]

In [62]:
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query
    result = get(query_url, headers=headers)
    json_result = json.loads(result.content)["artists"]["items"]
    
    if len(json_result) == 0:
            print("N/A")
            return None

    return json_result[0]

In [63]:
def get_songs_by_artists(token, artist_id, artist_name):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]

    track_ids = [track["id"] for track in json_result]
    audio_features_url = "https://api.spotify.com/v1/audio-features"
    audio_features_query = "?ids=" + ",".join(track_ids)
    audio_features_headers = get_auth_header(token)
    audio_features_result = get(audio_features_url + audio_features_query, headers=audio_features_headers)
    audio_features_json_result = json.loads(audio_features_result.content)["audio_features"]
    audio_features_map = {audio_feature["id"]: audio_feature for audio_feature in audio_features_json_result}

    track_data = []
    for idx, track in enumerate(json_result):
        track_id = track["id"]
        audio_feature = audio_features_map.get(track_id)
        if audio_feature:
            track.update(audio_feature)
        track_data.append({
            "Name": track['name'],
            "Popularity": track['popularity'],
            "Danceability": track['danceability'],
            "Energy": track['energy'],
            "Key": track['key'],
            "Loudness": track['loudness'],
            "Speechiness": track['speechiness'],
            "Acousticness": track['acousticness'],
            "Instrumentalness": track['instrumentalness'],
            "Liveness": track['liveness'],
            "Valence": track['valence'],
            "Tempo": track['tempo']
        })

    df = pd.DataFrame(json_result)
    df = df.loc[:, ['duration_ms', 'name', 'popularity', 'preview_url', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness','instrumentalness', 'acousticness', 'liveness', 'valence', 'tempo', 'time_signature']]
    df['artist'] = artist_name
    column_order = ['artist', 'name', 'popularity', 'key', 'mode', 'tempo', 'time_signature', 'duration_ms', 'danceability', 'energy', 'loudness', 'speechiness', 'instrumentalness', 'acousticness', 'liveness', 'valence', 'preview_url']
    df = df.reindex(columns=column_order)

    file_path = r'C:\Users\onee5\Desktop\Python Projects\Spotify\outputs\{}.csv'.format(artist_name)
    df.to_csv(file_path, index=False)
    print(f"Data saved to {file_path}")

In [64]:
token = get_token()
artist_name = "THE BEATLES"
result = search_for_artist(token, artist_name)
artist_id = result["id"]
get_songs_by_artists(token, artist_id, artist_name)

Data saved to C:\Users\onee5\Desktop\Python Projects\Spotify\outputs\THE BEATLES.csv
